In [1]:
from huggingface_hub import login
access_token_write = "your hugging face access token"
login(token = access_token_write)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/id2531/.cache/huggingface/token
Login successful


# IMPORTS

## PACKAGES

In [2]:
from langchain_community.vectorstores import Chroma
from chromadb.utils.embedding_functions import DefaultEmbeddingFunction
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from sklearn import metrics
import os
import re
import numpy as np
import pandas as pd
import chromadb
import transformers
import torch
import torch_xla.core.xla_model as xm
import pickle as pkl

## DEVICE

In [3]:
# device = 
# device

## MODEL

In [3]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float32,
    device_map="auto",
)

2024-01-15 16:13:21.205852: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 16:13:21.205895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 16:13:21.206992: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 16:13:23.340454: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# FUNCTIONS DEFINITION

## PROMPT EXAMPLES
f"""Consider three examples [EXAMPLE 1], [EXAMPLE 2], [EXAMPLE 3], instances of narratives from NASA's ASRS database labeled as {label}". Your objective is to assess if a [NEW NARRATIVE] can be assigned the same label as these examples. Return 1 if it can, and 0 otherwise."""

f"""Given threes examples [EXAMPLE 1], [EXAMPLE 2], [EXAMPLE 3] which are instances of narratives from NASA's ASRS database labeled as "{label}", your task is to determine whether a [NEW NARRATIVE] can be assigned the same label as these examples. Return 1 if the [NEW NARRATIVE] can be assigned the same label as the examples and 0 otherwise."""

## FUNCTIONS

In [5]:
def labeller(**kwargs):
    """For each Anomaly, return {Anomaly:1} and {Others:1} if the cell is empty.
    """
    for name, item in kwargs.items():
        if isinstance(item, str):
            cell = item
        if 'anomalies' in name.lower() and isinstance(item, list):
            anomalies_list = item
    cell_labels =[]
    for prefix in anomalies_list:
        if any(anomaly.strip().startswith(prefix) for anomaly in cell.split(';')):
            cell_labels.append(prefix)
    if cell_labels == []:
        cell_labels = ['Others']
    metadata = dict(zip(cell_labels, [1]*len(cell_labels)))
    return metadata 


def load_from_df(**kwargs):
    """return the narratives and their Anomalies in two list.
    """
    for name, item in kwargs.items():
        if "docs" in name.lower():
            source = item
        if "metadata" in name.lower():
            meta = item
        if isinstance(item, pd.DataFrame):
            data = item
        if 'anomalies' in name.lower() and isinstance(item, list):
            anomalies_list = item
    documents = data[source].apply(lambda cell: cell.strip()).values.tolist()
    metadata = data[meta].apply(lambda cell: labeller(input=cell, anomalies_list=anomalies_list)).values.tolist()
    return documents, metadata

def get_prompt(**kwargs):
    """Few shot prompt builder"""
    for name, item in kwargs.items():
        if 'example_1' in name.lower():
            example_1 = item
        if 'example_2' in name.lower():
            example_2 = item
        if 'example_3' in name.lower():
            example_3 = item
        if 'narrative' in name.lower():
            narrative = item
        if 'label' in name.lower():
            label = item
    system_prompt = f"""Consider three examples [EXAMPLE 1], [EXAMPLE 2], [EXAMPLE 3], instances of narratives from NASA's ASRS database labeled as {label}". Your objective is to assess if a [NEW NARRATIVE] can be assigned the same label as these examples. As [Answer] return 1 if it can, and 0 otherwise."""  #  Only provide the numerical result (0 or 1) and no additional information.
    full_prompt = f"""[INST]
    <<SYS>>
    {system_prompt}
    <</SYS>>
    
    [EXAMPLE 1] :
    {example_1}
    
    [EXAMPLE 2] :
    {example_2}

    [EXAMPLE 3] :
    {example_3}
    
    [NEW NARRATIVE] :
    {narrative}
    
    [/INST][Answer]"""
    return full_prompt

def format_pred(**kwargs):
    """"""
    for name, item in kwargs.items():
        if "inference" in name.lower():
            inference = item
        if "pattern" in name.lower():
            pattern = item
    pred = np.nan
    if pattern in inference[0]['generated_text'].strip():
        pred = 1
    else:
        pred = 0
    return pred, inference
            

def get_inference(**kwargs):
    """"""
    for name, item in kwargs.items():
        if "query" in name.lower():
            query = item.strip()
        if "pattern" in name.lower():
            pattern = item.strip()
        if "task" in name.lower():
            task = item.strip()
        if isinstance(item,
                      chromadb.api.models.Collection.Collection):
            store = item
        if isinstance(item,
                      transformers.pipelines.text_generation.TextGenerationPipeline):
            pipeline = item
    examples = store.query(query_texts=[query],
                          include=["documents"],
                          where={task:1},
                          n_results=3)['documents'][0]
    prompt = get_prompt(**dict(narrative=query,
                               example_1=examples[0],
                               example_2=examples[1],
                               example_3=examples[2],
                               label=task))
    inference = pipeline(prompt,
                         temperature=0.01,
                         do_sample=True,
                         num_return_sequences=1,
                         eos_token_id=tokenizer.eos_token_id,
                         max_length=4000)
    pred = format_pred(inference=inference, pattern=pattern)
    return pred

In [6]:
def binary_accuracy_per_label(y_true, y_pred):
    correct = y_pred == y_true
    accuracy_per_label = correct.float().mean(axis=0)
    return accuracy_per_label

def binary_accuracy_averaged(y_true, y_pred):
    accuracy_per_label = binary_accuracy_per_label(y_true, y_pred)
    accuracy_averaged = accuracy_per_label.mean()
    return accuracy_averaged

def custom_classification_report(y_true, y_pred):
    report = metrics.classification_report(y_true, y_pred, output_dict=True, target_names=ANOMALY_LABELS, zero_division=0)
    accuracy = binary_accuracy_per_label(y_true, y_pred)
    extended_accuracy_new = np.append(accuracy, [accuracy.mean()] * (len(report) - len(accuracy)))

    updated_report = {}
    for i, class_label in enumerate(report.keys()):
        # Create a new dictionary for the class with binary accuracy
        class_dict = {'binary_accuracy': extended_accuracy_new[i]}
        
        # Merge this dictionary with the existing metrics for the class
        class_dict.update(report[class_label])

        # Update the main report dictionary
        updated_report[class_label] = class_dict

    return updated_report

In [7]:
def zero_shot_prompt(**kwargs):
    """Zero shot prompt builder"""
    for name, item in kwargs.items():
        if 'narrative' in name.lower():
            narrative = item
    system_prompt = """<<SYS>> You will be provided with an input, which is the narrative from Aviation Safety reports of the NASA's ASRS dataset. Tell me if the input corresponds to which anomalies among the following categories: 'Deviation / Discrepancy - Procedural', 'Aircraft Equipment', 'Conflict', 'Inflight Event / Encounter', 'ATC Issue', 'Deviation - Altitude', 'Deviation - Track / Heading', 'Ground Event / Encounter', 'Flight Deck / Cabin / Aircraft Event', 'Ground Incursion', 'Airspace Violation', 'Deviation - Speed', 'Ground Excursion', 'No Specific Anomaly Occurred'. The input can correspond to one or many of them. Only return the corresponding anomalies.<<SYS>>"""
    full_prompt = "[INST]" + system_prompt + "\n" + narrative  + "[/INST]"
    return full_prompt

def format_zs_pred(**kwargs):
    """Parse the output of the text generation pipeline in a one-hot encoding list as predictions.
    """
    for name, item in kwargs.items():
        if "inference" in name.lower():
            inference = item
        if "anomalies" in name.lower():
            anomalies_list = item
    pattern = re.compile(r'\[INST\].*?\[/INST\]', re.DOTALL)
    inference = re.sub(pattern, '', inference[0]['generated_text'].strip())
    pred = [1 if label in inference else 0 for label in anomalies_list]
    return pred
    
def zero_shot_inference(**kwargs):
    """Run inference given a narrative and return a one-hot list of present anomalies in the narrative.
    """
    for name, item in kwargs.items():
        if "narrative" in name.lower():
            narrative = item.strip()
        if "pattern" in name.lower():
            pattern = item.strip()
        if "anomalies" in name.lower():
            anomalies = item
        if isinstance(item,
                      transformers.pipelines.text_generation.TextGenerationPipeline):
            pipeline = item
    prompt = zero_shot_prompt(**dict(narrative=narrative))
    
    inference = pipeline(prompt,
                         do_sample=True,
                         temperature=0.01,
                         top_k=10,
                         num_return_sequences=1,
                         eos_token_id=tokenizer.eos_token_id,
                         max_length=2000)
    pred = format_zs_pred(inference=inference, anomalies=anomalies)
    return pred
    
    

In [8]:
ANOMALY_LABELS = ['Deviation / Discrepancy - Procedural',
                    'Aircraft Equipment',
                    'Conflict',
                    'Inflight Event / Encounter',
                    'ATC Issue',
                    'Deviation - Altitude',
                    'Deviation - Track / Heading',
                    'Ground Event / Encounter',
                    'Flight Deck / Cabin / Aircraft Event',
                    'Ground Incursion',
                    'Airspace Violation',
                    'Deviation - Speed',
                    'Ground Excursion',
                    'No Specific Anomaly Occurred']

In [9]:
def encode_cell(cell: pd.Series, labels: list) -> pd.Series:
    """Encode the multilabels cell such that the cell content is replaced by \n
    a list of same length as labels and containing 0/1.

    Args:
        cell (pd.Series): cell containing the multilabel target
        labels (list): actual list of labels to classify.

    Returns:
        pd.Series: Expand of the cell with number of cols\n
        equal to number of element in labels.
    """
    cell_anomalies = [item.strip() for item in cell.split(';')]
    splited_cell_anomalies = {label: any(item.startswith(label)
                                         for item in cell_anomalies)
                              for label in labels}
    return pd.Series(splited_cell_anomalies)


def target_encoder(**kwargs):
    """_summary_
    Args:
        kwargs:
            Datasets: kwargs should key-value of the datasets
            target(str): All the passed datasets should contain this column.
            That's the column to encode.
            labels(list): list of the labels to encode.
    """
    data_list = []
    for name, item in kwargs.items():
        if "target" in name.lower():
            target = item
        if "labels" in name.lower():
            labels = tuple(item) if isinstance(item, list) else item
    for name, item in kwargs.items() :
        if isinstance(item, pd.DataFrame):
            data = item.copy()
            encoding_series = data[target].apply(
                lambda cell: encode_cell(cell, labels))
            data[target] = encoding_series.values.tolist()
            data_list.append(data)
    return data_list


# LOAD DATA

In [10]:
%%time
train_data_pkl = pd.read_pickle("./data/train_data_final.pkl")[0]  # load pkl data
test_data_pkl = pd.read_pickle("./data/test_data_final.pkl")[0]  # load pkl test data
train_data = train_data_pkl[['Narrative', 'Anomaly']].dropna(axis=0, subset=['Narrative','Anomaly'])
test_data = test_data_pkl[['Narrative', 'Anomaly']].dropna(axis=0, subset=['Narrative','Anomaly'])
train_data_enc, test_data_enc = target_encoder(train=train_data, test=test_data, target='Anomaly', labels=ANOMALY_LABELS)

CPU times: user 25.2 s, sys: 1.55 s, total: 26.7 s
Wall time: 25.5 s


# CONTENT

## SETUP THE VECTORE STORE

In [16]:
asrsnlp_client = chromadb.PersistentClient(path="~/chromadb/asrsnlp")

In [ ]:
try:
    asrsnlp_collection = asrsnlp_client.get_collection("asrsnlp_collecion")
except ValueError:
    embedder = DefaultEmbeddingFunction()
    asrsnlp_collection = asrsnlp_client.get_or_create_collection(
    name="asrsnlp_collection",
    metadata={"hnsw:space": "cosine"},
    embedding_function=embedder)
    documents = load_from_df(df=train_data, docs='Narrative',metadata='Anomaly', anomalies=ANOMALY_LABELS)

    asrsnlp_collection.add(
        documents=documents[0][:40000],
        metadatas=documents[1][:40000],
        ids=[f"ID{i}" for i in range(0,40000)]
    )

    asrsnlp_collection.add(
        documents=documents[0][40000:80000],
        metadatas=documents[1][40000:80000],
        ids=[f"ID{i}" for i in range(40000, 80000)]
    )

    asrsnlp_collection.add(
        documents=documents[0][80000:],
        metadatas=documents[1][80000:],
        ids=[f"ID{i}" for i in range(80000,96986)]
    )

# INFERENCE

## ZERO SHOT

In [43]:
%%time
# SINGLE INFERENCE
result_zs = zero_shot_inference(narrative=test_data.Narrative[2],
                             anomalies=ANOMALY_LABELS,
                             pipeline=pipeline)

CPU times: user 34min 20s, sys: 1.57 s, total: 34min 21s
Wall time: 52.3 s


### EVALUATION

In [18]:
# INFERENCE ON ALL DATA IN THE TEST DATA
preds_zs = test_data_enc.head(20).Narrative.apply(lambda cell: zero_shot_inference(narrative=cell, anomalies=ANOMALY_LABELS, pipeline=pipeline))

In [27]:
ytrue = test_data_enc.head(20).Anomaly.apply(lambda cell: [1 if b else 0 for b in cell])
ytrue = np.array(ytrue.values.tolist())[:,:-1].tolist()

## FEW SHOT

In [ ]:
%%time
# SINGLE INFERENCE
result_fs = get_inference(query=test_data.Narrative[1],
                       task='No Specific Anomaly Occurred',
                       pattern="[Answer]  Yes",
                       store=asrsnlp_collection,
                       pipeline=pipeline)

### EVALUATION

In [ ]:
# INFERENCE ON ALL DATA IN THE TEST DATA
preds_fs = test_data.head(20).Narrative.apply(lambda cell: get_inference(query=cell,
                                                                      task='No Specific Anomaly Occurred',
                                                                      pattern="[Answer]  Yes",
                                                                      narrative=cell,
                                                                      store=asrsnlp_collection,
                                                                      pipeline=pipeline))

# OTHERS

text_classif_pipeline = transformers.pipeline(
    "text-classification",
    model=model,
    torch_dtype=torch.float32,
    device_map="auto",
)